# Aphrodisias analysis notebook


In [1]:
from pathlib import Path

import pygeoprocessing
from osgeo import gdal
import statistics
import math
import numpy as np
import rasterio as rio
from rasterio.windows import from_bounds
from shapely.geometry import Polygon 


### Defining input datasets

In [3]:
gis_folder = Path(r"C:\Users\lizad\OneDrive\Desktop\Brown\Dissertation\GIS")

dem_raster_path = gis_folder / "DEM_UTM_35N.tif"
dem_raster_info = pygeoprocessing.get_raster_info(str(dem_raster_path))

slope_raster_path = gis_folder / "slope_35N.tif"
tri_raster_path = gis_folder / "tri_35N.tif"
landcover_raster_path = gis_folder / "landcover_35N.tif"

church_vector_path = gis_folder / "churches_35N.gpkg"
city_center_vector_path = gis_folder / "center_35N.gpkg"

church_raster_path = gis_folder / "churches_35N.tif"
city_center_raster_path = gis_folder / "center_35N.tif"

### Rasterizing Churches to match the DEM

In [3]:
# Create and burn church locations
if not church_raster_path.exists:
    pygeoprocessing.new_raster_from_base(
        str(dem_raster_path),
        str(church_raster_path),
        gdal.GDT_Byte,
        [0],
    )
    pygeoprocessing.rasterize(
    str(church_vector_path),
    str(church_raster_path),
    [1],
    )

### Rasterizing city center to match the DEM

In [4]:
# Create and burn city center
if not city_center_raster_path.exists:
    pygeoprocessing.new_raster_from_base(
        str(dem_raster_path),
        str(city_center_raster_path),
        gdal.GDT_Byte,
        [0],
    )
    pygeoprocessing.rasterize(
    str(city_center_vector_path),
    str(city_center_raster_path),
        [1],
    )

### Calculate slope and Terrain Ruggedness Index from DEM

In [5]:
slope_gdal_ds = gdal.DEMProcessing(str(slope_raster_path), str(dem_raster_path), "slope")
del slope_gdal_ds

# pygeoprocessing.calculate_slope((str(dem_raster_path), 1), str(slope_raster_path))

tri_gdal_ds = gdal.DEMProcessing(str(tri_raster_path), str(dem_raster_path), "tri")
del tri_gdal_ds

### Normalize input datasets

In [ ]:
#Align and clip landscape data to DEM extent
dem_raster_info = pygeoprocessing.get_raster_info(str(dem_raster_path))
dem_raster_clip = gis_folder / "DEM_UTM_35N_clip.tif"
landcover_raster_clip = gis_folder / "landcover_35N_clip.tif"
slope_raster_clip = gis_folder / "slope_35N_clip.tif"
tri_raster_clip = gis_folder / "tri_35N_clip.tif"

pygeoprocessing.align_and_resize_raster_stack(
    [str(dem_raster_path), str(landcover_raster_path), str(slope_raster_path), str(tri_raster_path)],
    [str(dem_raster_clip), str(landcover_raster_clip), str(slope_raster_clip), str(tri_raster_clip)],
    ["near", "near"],
    dem_raster_info["pixel_size"],
    dem_raster_info["bounding_box"],
    raster_align_index=0,
)

### Reclassify landcover into cost surface

### Calculate Tobler's hiking function

In [44]:
tobler_surface = gis_folder / "tobler_surface.tif"

slope_raster_info = pygeoprocessing.get_raster_info(str(slope_raster_path))
slope_cell_resolution = statistics.mean([abs(x) for x in slope_raster_info["pixel_size"]])

slope_array = pygeoprocessing.raster_to_numpy_array(str(slope_raster_path))

# def tobler_op(slope):
#     result = (slope_cell_resolution/1000)/(6*math.exp(-3.5*abs(math.tan(slope*math.pi/180)+0.05)))
#     return result
def tobler_op(slope):
    result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(slope+0.05)))
    return result
tobler_op_vectorized = np.vectorize(tobler_op)
# test_array = tobler_op_vectorized(slope_array)

pygeoprocessing.raster_map(
    tobler_op,
    [str(slope_raster_path)],
    str(tobler_surface),
)

C:\Users\lizad\AppData\Local\Temp\ipykernel_111576\3273824899.py:12: RuntimeWarning: divide by zero encountered in divide
  result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(slope+0.05)))
C:\Users\lizad\AppData\Local\Temp\ipykernel_111576\3273824899.py:12: RuntimeWarning: overflow encountered in divide
  result = (slope_cell_resolution/1000)/(6*np.exp(-3.5*np.abs(slope+0.05)))


### Calculate combined friction surface using raster calculator

In [ ]:
friction_surface = gis_folder / "friction_surface.tif"

file_list = [
    str(friction_surface),
    str(friction_surface_1),
    str(friction_surface_2)
]

def friction_op(slope, vegetation):
    result = slope*0.8 + vegetation*2
    return result

if not friction_surface.exists():
    pygeoprocessing.raster_calculator(
        file_list,
        friction_op,
        str(friction_surface),
        gdal.GDT_Float32,
        -1,
    )